In [91]:
%matplotlib inline

import numpy as np
import numpy
import pandas as pd
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

In [92]:
import imblearn
print(str(imblearn.__version__))

0.4.3


In [93]:
#setting the seed for reproducibility
#seed = 10110
seed = 12061204
numpy.random.seed(seed)

#load the dataset
dataframe = pandas.read_csv("./pubchem_data/processed/pparg_ligand_data.txt", sep="\t")
from sklearn.utils import shuffle
dataframe = shuffle(dataframe)

In [94]:
mols = []
fps = []

#get molecules and then get fingerprints from those
for index, row in dataframe.iterrows():
    mol = Chem.MolFromSmiles(row['SMILES'])
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2)
    mols.append(mol)
    fps.append(fp)

#Convert the RDKit vectors into numpy arrays
#Based on: http://www.rdkit.org/docs/Cookbook.html#using-scikit-learn-with-rdkit
np_fps = []
for fp in fps:
    arr = numpy.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    np_fps.append(arr)

np_fps_array = numpy.array(np_fps)

In [95]:
#Need to encode my classes
#Ligand = 0, not_ligand = 1
encoder = LabelEncoder()
encoder.fit(dataframe['Type'])
enc_y = encoder.transform(dataframe['Type'])

In [96]:
data = np_fps_array #전체 데이터
targets = np.asarray(enc_y)# 전체 타겟.

all_index = np.arange(data.shape[0], dtype = np.int32) #전체 인덱스 생성
np.random.shuffle(all_index) #인덱스 섞기
n = int(np_fps_array.shape[0] * 0.8) #몇퍼센트로 나눌거냐?

train_index = all_index[:n]
test_index = all_index[n:]

In [97]:
train_data = data[train_index]
test_data = data[test_index]

train_targets = targets[train_index]
test_targets = targets[test_index]

In [98]:
n1 = int(len(train_index) * 0.2)
n2 = int(len(train_index) * 0.4)
n3 = int(len(train_index) * 0.6)
n4 = int(len(train_index) * 0.8)

val_index = train_index[0:n1]
train_index = train_index[n1:]


In [99]:
def build_model():
    model = Sequential()
    model.add(Dense(2048, input_dim=2048, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


sm = SMOTE(random_state=12, ratio = 'minority')
x_train_res, y_train_res = sm.fit_sample(train_data, train_targets)

train_data, train_targets = x_train_res, y_train_res 


In [ ]:
for i in range(5) : 
    t_i = train_index[i*:]

In [13]:
estimator = KerasClassifier(build_fn=build_model,epochs=5,batch_size=5)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
y_pred = cross_val_predict(estimator, x_train_res, y_train_res, cv=kfold)
conf_mat = confusion_matrix(y_train_res, y_pred)
conf_mat

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
982/982 [==============================] - 3s 4ms/step - loss: 0.0912 - acc: 0.9674
Epoch 2/5
982/982 [==============================] - 1s 1ms/step - loss: 0.0091 - acc: 0.9959
Epoch 3/5
982/982 [==============================] - 1s 1ms/step - loss: 0.0042 - acc: 0.9990
Epoch 4/5
982/982 [==============================] - 1s 1ms/step - loss: 7.0965e-06 - acc: 1.0000
Epoch 5/5
982/982 [==============================] - 1s 1ms/step - loss: 4.8785e-06 - acc: 1.0000
Epoch 1/5
982/982 [==============================] - 2s 2ms/step - loss: 0.1212 - acc: 0.9501
Epoch 2/5
982/982 [==============================] - 1s 1ms/step - loss: 8.0085e-04 - acc: 1.0000
Epoch 3/5
982/982 [==============================] - 1s 1ms/step - loss: 3.7416e-05 - acc: 1.0000
Epoch 4/5
982/982 [==============================] - 1s 1ms/step - loss: 1.7015e-05 - acc: 1.0000
Epoch 5/5
982/

array([[614,   0],
       [  4, 610]], dtype=int64)

In [11]:


k = 5
num_val_samples = len(x_train_res) // k
num_epochs = 5
all_mae_histories = []
sum_mat = [[0,0],[0,0]] #conf_mat 더할 배열.


for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    v_i = train_index[i * num_val_samples: (i + 1) * num_val_samples]
    
    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = x_train_res[np.concatenate(
        [train_index[:i * num_val_samples],
         train_index[(i + 1) * num_val_samples:]],
        axis=0)]
    partial_train_targets = y_train_res[np.concatenate(
        [train_index[:i * num_val_samples],
         train_index[(i + 1) * num_val_samples:]],
        axis=0)]

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=5)
    pred_y = model.predict(x_train_res[v_i])
    i = 0
    for k in pred_y :
        if(k<0.5) : pred_y[i] = 0 #왜 여기서는 이렇게 되는가?? 이미 y_test 가 인코딩이 되었기 때문에!!(그니까 답안지가 바뀐거!) 
        else : pred_y[i] = 1
        i += 1
    conf_mat = confusion_matrix(y_train_res[v_i], pred_y)
    print(conf_mat)
    sum_mat += conf_mat
print(sum_mat)

처리중인 폴드 # 0


NameError: name 'val_data' is not defined

In [ ]:
y_train_res[v_i].shape

In [ ]:
val_targets.shape

In [70]:
model = build_model()
model.save_weights('model.h5') #초기 가중치 저장하기

    #cross validation 구현.
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed ) # Define the split - into 2 folds 
kf.get_n_splits(x_train_res) # returns the number of splitting iterations in the cross-validator
validation_acc_list = [0,0,0,0,0] # 각 단계에서 val_acc 저장할 배열
num = 0
sum_mat = [[0,0],[0,0]] #conf_mat 더할 배열.
    
for train_index, test_index in kf.split(x_train_res, y_train_res):
    x_train, x_test = x_train_res[train_index], x_train_res[test_index]
    y_train, y_test = y_train_res[train_index], y_train_res[test_index]
    model.load_weights('model.h5') #가중치 초기화
    model.fit(x_train,y_train, epochs =5, batch_size=5)
#         scores = model.evaluate(x_test,y_test)
#         validation_acc_list[num] = scores[1]*100
#         print(validation_acc_list[num])
    pred_y = model.predict(x_test)
    pred_y[0]
    i = 0
    for k in pred_y :
        if(k<0.5) : pred_y[i] = 0 #왜 여기서는 이렇게 되는가?? 이미 y_test 가 인코딩이 되었기  때문에(그니까 답안지가 바뀐거!)!! 
        else : pred_y[i] = 1
        i += 1
    conf_mat = confusion_matrix(y_test,pred_y)
    print(conf_mat)
    sum_mat = sum_mat + conf_mat
    print(sum_mat)
    num+=1


Epoch 1/5
400/400 [==============================] - 2s 5ms/step - loss: 0.3667 - acc: 0.9525
Epoch 2/5
400/400 [==============================] - 1s 1ms/step - loss: 0.0949 - acc: 0.9625
Epoch 3/5
400/400 [==============================] - 1s 1ms/step - loss: 0.0167 - acc: 0.9950
Epoch 4/5
400/400 [==============================] - 1s 1ms/step - loss: 0.0134 - acc: 0.9950
Epoch 5/5
400/400 [==============================] - 1s 1ms/step - loss: 0.0056 - acc: 0.9975
[[ 0  4]
 [ 0 96]]
[[ 0  4]
 [ 0 96]]
Epoch 1/5
400/400 [==============================] - 1s 1ms/step - loss: 0.2249 - acc: 0.9575
Epoch 2/5
400/400 [==============================] - 1s 1ms/step - loss: 0.0561 - acc: 0.9700
Epoch 3/5
400/400 [==============================] - 1s 1ms/step - loss: 0.0077 - acc: 1.0000
Epoch 4/5
400/400 [==============================] - 1s 1ms/step - loss: 3.0638e-04 - acc: 1.0000
Epoch 5/5
400/400 [==============================] - 1s 1ms/step - loss: 1.5617e-04 - acc: 1.0000
[[ 2  2]
 [ 5 

In [15]:
model.fit(x_train_res,y_train_res, epochs = 5)

# sm = SMOTE(random_state=12, ratio = 'minority')
# test_data, test_targets = sm.fit_sample(test_data, test_targets)

pred_y = model.predict(test_data)
pred_y[0]
i = 0
for k in pred_y :
    if(k<0.5) : pred_y[i] = 0 #왜 여기서는 이렇게 되는가?? 이미 y_test 가 인코딩이 되었기  때문에(그니까 답안지가 바뀐거!)!! 
    else : pred_y[i] = 1
    i += 1
conf_mat = confusion_matrix(test_targets,pred_y)
print(conf_mat)


Epoch 1/5
1228/1228 [==============================] - 0s 238us/step - loss: 0.0141 - acc: 0.9967
Epoch 2/5
1228/1228 [==============================] - 0s 201us/step - loss: 2.2123e-04 - acc: 1.0000
Epoch 3/5
1228/1228 [==============================] - 0s 201us/step - loss: 9.5096e-05 - acc: 1.0000
Epoch 4/5
1228/1228 [==============================] - 0s 201us/step - loss: 6.8179e-05 - acc: 1.0000
Epoch 5/5
1228/1228 [==============================] - 0s 202us/step - loss: 5.4147e-05 - acc: 1.0000
[[  0  12]
 [  0 148]]


In [16]:
train_targets.tolist().count(0)

22

In [32]:
test_targets.tolist().count(0)

12

왜 따로 빼 놓으면 못 맞추는데?

In [ ]:
list_train = []
list_test = []
for train_index, test_index in kf.split(x_train_res, y_train_res):
    list_train.append(train_index)
    list_test.append(test_index)
    print()
    print("==================================================================================")
#심각한데 ?? 학습이 되버렸네?

In [ ]:
list_train[0]

In [ ]:
list_train[2]

In [84]:
data_tmp = train_data
target_tmp = train_targets

In [90]:
data_tmp

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.14007162, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [100]:

val_data = data_tmp[900:]
train_data = data_tmp[:900]
val_targets = target_tmp[900:]
train_targets = target_tmp[:900]

In [101]:
model = build_model()
model.fit(train_data, train_targets, epochs=5)

pred_y = model.predict(val_data)
i = 0
for k in pred_y :
    if(k<0.5) : pred_y[i] = 0 #왜 여기서는 이렇게 되는가?? 이미 y_test 가 인코딩이 되었기  때문에(그니까 답안지가 바뀐거!)!! 
    else : pred_y[i] = 1
    i += 1
conf_mat = confusion_matrix(val_targets,pred_y)
print(conf_mat)

Epoch 1/5
900/900 [==============================] - 2s 2ms/step - loss: 0.1974 - acc: 0.9067
Epoch 2/5
900/900 [==============================] - 0s 227us/step - loss: 0.0111 - acc: 0.9956
Epoch 3/5
900/900 [==============================] - 0s 231us/step - loss: 8.8209e-04 - acc: 1.0000
Epoch 4/5
900/900 [==============================] - 0s 230us/step - loss: 1.3857e-04 - acc: 1.0000
Epoch 5/5
900/900 [==============================] - 0s 227us/step - loss: 5.9095e-05 - acc: 1.0000
[[318]]


In [109]:
for i in x_train_res[1200] : 
    print(i)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.37561011809073996
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.62438988190926
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.37561011809073996
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [57]:
pred_y

[]

In [38]:
train_data.shape

(636, 2048)